### Étape 1 – Connexion Python

In [2]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility
import pandas as pd
import numpy as np

# === Étape 1 : Connexion à Milvus ===
connections.connect("default", host="localhost", port="19530")
print(" Connecté à Milvus")

# === Étape 2 : Définition du schéma ===
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="title", dtype=DataType.VARCHAR, max_length=512),
    FieldSchema(name="artist", dtype=DataType.VARCHAR, max_length=256),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=384),
]

schema = CollectionSchema(fields, description="Embeddings musicaux GutsOfDarkness")

collection_name = "music_embeddings"

# Supprime la collection si elle existe déjà (utile pour tester plusieurs fois)
existing_collections = utility.list_collections()
if collection_name in existing_collections:
    collection = Collection(collection_name)
    collection.drop()
    print(f" Ancienne collection '{collection_name}' supprimée.")

# Création de la collection
collection = Collection(collection_name, schema=schema)
print(" Collection créée :", collection.name)

# === Étape 3 : Lecture du dataset ===
df = pd.read_parquet("../data/processed/sample_albums_embedded.parquet")
print(f" {len(df)} lignes chargées depuis le fichier Parquet.")

# === Étape 4 : Conversion des embeddings ===
# Si ton embedding est stocké sous forme de chaîne (ex: "[0.1, 0.2, ...]")
if isinstance(df["embedding"].iloc[0], str):
    df["embedding"] = df["embedding"].apply(
        lambda x: np.array(eval(x), dtype=np.float32)
    )

# Vérifie la taille du premier vecteur
print(" Taille du premier embedding :", len(df["embedding"].iloc[0]))

# === Étape 5 : Insertion dans Milvus ===
collection.insert([
    df["album_name"].tolist(),
    df["artist_name"].tolist(),
    df["embedding"].tolist()
])

collection.flush()
print(f"✅ {len(df)} morceaux insérés dans Milvus.")


 Connecté à Milvus
 Ancienne collection 'music_embeddings' supprimée.
 Collection créée : music_embeddings
 14072 lignes chargées depuis le fichier Parquet.
 Taille du premier embedding : 384
✅ 14072 morceaux insérés dans Milvus.


### TEST

In [5]:
from sentence_transformers import SentenceTransformer

# Charger le modèle d'embedding (même que celui utilisé pour les albums)
model = SentenceTransformer("all-MiniLM-L6-v2")

# Exemple : une requête utilisateur
query = "album de dark ambient expérimental et introspectif"

# Transformer la requête en vecteur
query_vec = model.encode([query]).astype(np.float32)

# === Création de l’index vectoriel (avant le load) ===
index_params = {
    "index_type": "HNSW",  # Algorithme de recherche approximative performant
    "metric_type": "IP",   # Inner Product (cosine-like similarity)
    "params": {"M": 8, "efConstruction": 64}
}
collection.create_index("embedding", index_params)
print("✅ Index créé avec succès.")

# Charger la collection après indexation
collection.load()
print("✅ Collection chargée en mémoire pour la recherche.")

# === Recherche vectorielle ===
search_params = {"metric_type": "IP", "params": {"nprobe": 10}}
results = collection.search(
    data=query_vec,
    anns_field="embedding",
    param=search_params,
    limit=5,
    output_fields=["title", "artist"]
)

# === Affichage des résultats ===
for hits in results:
    for hit in hits:
        print(f"🎧 {hit.entity.get('title')} — {hit.entity.get('artist')} (score: {hit.distance:.3f})")


✅ Index créé avec succès.
✅ Collection chargée en mémoire pour la recherche.
🎧 dark matters — k. meizter (score: 0.655)
🎧 night chorus — harvest rain (score: 0.639)
🎧 endogenous — detritus (score: 0.615)
🎧 innerwar — brighter death now (score: 0.596)
🎧 songs from neuropa — mental measuretech (score: 0.584)
